# Version 1. 전부 다같이 학습

In [1]:



from sklearn.feature_extraction.text import TfidfVectorizer
import operator
import pandas as pd
import re
from konlpy.tag import Okt
okt=Okt()
import warnings
warnings.simplefilter("ignore")


#데이터 알맞은 형태로 전처리
def preprocess(documents):      
# 저장된 데이터에서 필요한건 '학교명', '학과명', '학과소개글', '배우는과목소개글'
# 따라서 얘네 4명만 살리고 학교명과 학과명은 하나로 합친 데이터정제 코드 
    documents = documents[['name','univname','text_intr','text_lect']]
    documents['name']=documents['univname']+' '+documents['name']
    documents = documents.drop(['univname'],axis=1)
    documents['text_intr']=documents['text_intr']+' '+documents['text_lect']
    documents = documents.drop(['text_lect'],axis=1)
    documents = documents.dropna(axis=0)
    documents=documents.reset_index()
    return documents





# 한글과 띄어쓰기를 제외한 모든 부분을 제거, 명사만 남김
def trans_hangul(documents):
    text_list = list(documents['text_intr'])
    hangul = re.compile('[^ ㄱ-ㅣ가-힣]+')
    for i in range(len(text_list)):   
        text_list[i] = hangul.sub('', text_list[i])
        text_list[i]=(' '.join(okt.nouns(text_list[i])))
        
    return text_list





def make_top100(documents,n):

# text_list_vec는 documents를 tfidf 벡터화한 벡터값(학습된 값)    
    text_list_vec = TfidfVectorizer().fit(documents)

# word_index는 각 단어의 고유 인덱스 값    
    word_index=text_list_vec.vocabulary_
    

    text_list_vec=text_list_vec.transform(documents).toarray()
    word_index = {v: k for k, v in word_index.items()}
    word_index= sorted(word_index.items(), key=operator.itemgetter(0))
    
# 이부분이 제일 중요!!!! 여기서 설정하는 값이 데이터의 n번째 학과의 top100을 도출시켜줌    
    index_of_major = n

    result={}
    for i in range(len(text_list_vec[index_of_major])):
        if text_list_vec[index_of_major][i] != 0:
            result[word_index[i][1]] =  text_list_vec[index_of_major][i]
    result= sorted(result.items(), key=operator.itemgetter(1),reverse=True)
    top100={}
    for i in range(100):
        top100[result[i][0]] = result[i][1]
        
    return top100



In [2]:

#####################main############################
# 현 코드에는 연습용데이터불러오는 코드임.  
#실무에선 실제 저장된 학과관련 데이터와 연결하는 코드를 써야함.
documents=pd.read_excel('example.xlsx')


#########이부분은 데이터를 훈련시켜서 각 학과마다 tdidf 돌리는 부분, 시간이 가장 오래걸릴 코드#########
x=preprocess(documents)
x=trans_hangul(x)



###훈련 다된 데이터에서 원하는 것만 뽑아와서 순서매기고 출력###
#받은 데이터에서 보고싶은 학과의 index입력(순서)
wanted_index = 4   
make_top100(x,wanted_index)



{'중국어': 0.6436017425317165,
 '중국': 0.5473780387607348,
 '문학': 0.11510215632399859,
 '문화': 0.11254997811566504,
 '중어': 0.11039871458716347,
 '중국문학': 0.10128580394003896,
 '현대문학': 0.09594971453370334,
 '이해': 0.09364667950554247,
 '한자': 0.08440483661669915,
 '한중': 0.08440483661669915,
 '편성': 0.08135419466165604,
 '능력': 0.08105922984335887,
 '중국인': 0.0790167424490227,
 '대한': 0.07866321078465567,
 '언어능력': 0.07483741192229264,
 '어문학': 0.07142265024308563,
 '작품': 0.07108893107420089,
 '문법': 0.06714292452233252,
 '독해력': 0.06603455607540919,
 '고급': 0.06411229332509145,
 '문장': 0.06366284364215481,
 '현대': 0.05498079942681256,
 '교육': 0.05350161373919628,
 '구사': 0.05282764486032735,
 '관련': 0.05223019563784956,
 '작가': 0.051916910795917995,
 '장르': 0.051916910795917995,
 '회화': 0.051916910795917995,
 '문학사': 0.05106284642971883,
 '중국학': 0.05064290197001948,
 '전문가': 0.049828732181505665,
 '언어학': 0.049484180438943294,
 '동시': 0.04658248621112291,
 '사회': 0.04585852606216824,
 '지식': 0.04540217284134643,
 '특색

# Version 2. 한 학과선택, 램덤한 개수의 학과와 비교해 tfidf학습

In [5]:
import random
from sklearn.feature_extraction.text import TfidfVectorizer
import operator
import pandas as pd
import re
from konlpy.tag import Okt
okt=Okt()
import warnings
warnings.simplefilter("ignore")
import time
start = time.time()  # 시작 시간 저장
 
 





#데이터 알맞은 형태로 전처리
# 저장된 데이터에서 필요한건 '학교명', '학과명', '학과소개글', '배우는과목소개글'
# 따라서 얘네 4명만 살리고 학교명과 학과명은 하나로 합친 데이터정제 코드
# 결론적적으로 데이터가 들어오면 '학교+학과' , '학과관련text'으로 return
def preprocess(documents):      
    documents = documents[['name','univname','text_intr','text_lect']]
    documents['name']=documents['univname']+' '+documents['name']
    documents = documents.drop(['univname'],axis=1)
    documents['text_intr']=documents['text_intr']+' '+documents['text_lect']
    documents = documents.drop(['text_lect'],axis=1)
    documents = documents.dropna(axis=0)
    documents=documents.reset_index(drop=True)
    return documents






# 선택한 학과와 그 외 학과를 랜덤으로 50개뽑아서 return
# n에는 원하는 학과의 인덱스를 입력
def random_choice(documents,n):
    df = pd.DataFrame( columns=['name', 'text_intr'])
    df=df.append(documents.loc[[n],:])
         
    x=[i for i in range(0,len(documents))]
    random.shuffle(x)
    x=x[:100]
    if n in x:
        x.remove(n)
    for i in x:
        df=df.append(documents.loc[[i],:])
        
    return df
        
        




# 한글과 띄어쓰기를 제외한 모든 부분을 제거, 명사만 남김
def trans_hangul(documents):
    text_list = list(documents['text_intr'])
    hangul = re.compile('[^ ㄱ-ㅣ가-힣]+')
    for i in range(len(text_list)):   
        text_list[i] = hangul.sub('', text_list[i])
        text_list[i]=(' '.join(okt.nouns(text_list[i])))
        
    return text_list






# 방대한 자료를 벡터화하는 함수(tfidf 훈련시키는 함수)
def make_top100(documents):
    global text_list_vec,word_index
    
# text_list_vec는 documents를 tfidf 벡터화한 벡터값(학습된 값)    
    text_list_vec = TfidfVectorizer().fit(documents)
# word_index는 각 단어의 고유 인덱스 값    
    word_index=text_list_vec.vocabulary_

    text_list_vec=text_list_vec.transform(documents).toarray()
    word_index = {v: k for k, v in word_index.items()}
    word_index = sorted(word_index.items(), key=operator.itemgetter(0))
  
    
# 학습된 tfidf를 바탕으로 중요도 높은 top100개를 도출

    result={}
    for i in range(len(text_list_vec[0])):
        if text_list_vec[0][i] != 0:
            result[word_index[i][1]] =  text_list_vec[0][i]
    result= sorted(result.items(), key=operator.itemgetter(1),reverse=True)
    top100={}
    for i in range(100):
        top100[result[i][0]] = result[i][1]
        
    return top100




#####################main############################
# 현 코드에는 연습용데이터불러오는 코드임.  
#실무에선 실제 저장된 학과관련 데이터와 연결하는 코드를 써야함.
print("time :", time.time() - start)  # 현재시각 - 시작시간 = 실행 시간

documents=pd.read_excel('majors_textSet_복구.xlsx')



# 알고 싶은 학과의 index를 wanted_index의 값으로 입력
wanted_index = 100



# 정리
# 5000개 기존데이터 입력 -> 데이터 중 필요한 부분만 남기도록 전처리 -> 원하는 학과(index로 선택)와 그 외 50개만 뽑음
# -> 51개 데이터 에서 한글과 띄어쓰기만 남김, 그리고 단어만 남김(조사 형용사 전부 삭제) 
# -> 총 51개를 tfidf학습시킴(서로 비교하면서 각 단어에 점수가 메겨짐)
# -> 원하는 학과 데이터 중에서 단어의 점수를 내림차순으로 정렬한 뒤 top100개만 뽑음



# 실제 실행 코드
print("time :", time.time() - start)  # 현재시각 - 시작시간 = 실행 시간

documents = preprocess(documents)

print("time :", time.time() - start)  # 현재시각 - 시작시간 = 실행 시간

print(documents['name'][wanted_index])
documents = random_choice(documents,wanted_index)

print("time :", time.time() - start)  # 현재시각 - 시작시간 = 실행 시간

documents =trans_hangul(documents)

print("time :", time.time() - start)  # 현재시각 - 시작시간 = 실행 시간


top100 = make_top100(documents)
print(top100)


 
print("time :", time.time() - start)  # 현재시각 - 시작시간 = 실행 시간


time : 0.0
time : 1.4940106868743896
time : 1.5628273487091064
국민대학교 공업디자인학과
time : 1.7353806495666504
time : 13.881948232650757
{'디자인': 0.7322765380540905, '제품': 0.2746818834777818, '조형': 0.22674575760616156, '환경': 0.1506627365937207, '공업': 0.12086002873022397, '산업': 0.11791843944890441, '학습': 0.11134492740153834, '제품디자인': 0.09796337184905783, '기능': 0.09498920846920518, '가능성': 0.0898870021987487, '형태': 0.08649963191766738, '사용자': 0.08502965910231058, '개발': 0.0799117775467422, '해결': 0.07972116051272439, '정보기기': 0.07598882117083529, '기획': 0.07485568819120879, '능력': 0.07478042046942987, '디자이너': 0.07382696211378532, '제안': 0.07382696211378532, '방법': 0.07112125073682193, '메카트로닉스': 0.07085804925192549, '재료': 0.07065325238389385, '관계': 0.06777847886466301, '바탕': 0.06565038529552794, '색채': 0.06530891456603856, '시스템': 0.0638156429399507, '표현': 0.062908823212849, '기법': 0.06093501797075269, '이해': 0.05861966622757901, '이론': 0.05687916556921176, '입체': 0.05462900796124181, '프로세스': 0.0537687579756266